In [1]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [34]:
%matplotlib inline
from sklearn.datasets import fetch_lfw_people
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import skimage
import skimage.io
from skimage.feature import hog
from sklearn.feature_extraction import image
from tqdm import tqdm, tqdm_notebook, tnrange
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.externals import joblib
test = 123
test

123

In [3]:
people = fetch_lfw_people(color=True)

In [4]:
people.keys()

people.images.shape

(13233, 62, 47, 3)

In [5]:
filepath = "C:\\Users\\danai\\Projects\\python projects\\PycharmProjects\\IT_1_coursework\\neg_ex\\космос.png"
file_ = skimage.io.imread(filepath)
patches = image.extract_patches_2d(file_, people.images.shape[1:3], max_patches = people.images.shape[0])

In [41]:
print(file_.shape)
print(patches.shape)

(1956, 3840, 3)
(13233, 62, 47, 3)


In [7]:
people_fv = list()
for img in tqdm_notebook(people['images'], desc='processing positive examples'):
    #visualise=True
    fv, hog_img = hog(img[:, :, 2], orientations=8, pixels_per_cell=(6, 6), cells_per_block=(1, 1), feature_vector=True, visualise=True)
    people_fv.append(fv)
    # print(fv.shape)
    # print(fv)
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
    ax1.axis('off')
    ax1.imshow(img.astype(np.int))
    ax1.set_title('Input image')
    ax2.axis('off')
    ax2.imshow(hog_img, cmap=plt.cm.gray)
    ax2.set_title('Histogram of Oriented Gradients')
    plt.show()
    """
people_fv=np.asarray(people_fv)

C:\ProgramData\Anaconda3\envs\cw_env\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [8]:
print(people_fv[0].shape)
print(len(people_fv))

(560,)
13233


In [9]:
patch_fv = list()
for patch in tqdm_notebook(patches, desc='processing negative examples'):
    #print(patch.shape)
    fv, hog_patch = hog(patch[:, :, 2], orientations=8, pixels_per_cell=(6, 6), cells_per_block=(1, 1), feature_vector=True, visualise=True)
    patch_fv.append(fv)
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
    ax1.axis('off')
    ax1.imshow(patch)
    ax1.set_title('Input image')
    ax2.axis('off')
    ax2.imshow(hog_patch,  cmap=plt.cm.gray)
    ax2.set_title('Histogram of Oriented Gradients')
    plt.show()
    """
patch_fv = np.asarray(patch_fv)

C:\ProgramData\Anaconda3\envs\cw_env\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [10]:
label=np.hstack((np.ones(len(people_fv)), np.zeros(len(patch_fv))))
all_ex=np.vstack((people_fv, patch_fv))
print(all_ex.shape)
print(label.shape)

X_train, X_test, y_train, y_test = train_test_split(all_ex, label, shuffle=True, random_state=42)#42-ответ на все вопросы

print(y_test.mean())
print(y_test.std())

(26466, 560)
[[0.03987536 0.47275938 0.21401908 ... 0.01881485 0.03012654 0.05858849]
 [0.01425075 0.18887777 0.2577391  ... 0.02433732 0.01549511 0.19706225]
 [0.25256104 0.62938581 0.06237439 ... 0.         0.         0.        ]
 ...
 [0.18072718 0.24772575 0.31622968 ... 0.05177573 0.07189899 0.03585996]
 [0.16530757 0.04975588 0.15271736 ... 0.13763868 0.2053392  0.19413811]
 [0.02678954 0.05374218 0.06365722 ... 0.01416995 0.05112067 0.18943369]]
(26466,)
[1. 1. 1. ... 0. 0. 0.]
0.5052138431313284
0.49997281510078323


In [11]:
cls = LinearSVC(random_state=42)#Автостопом по галактике
cls.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [23]:
x_pred=cls.predict(X_test)
x_pred

array([0., 0., 1., ..., 1., 0., 0.])

In [50]:
print(f"score: {cls.score(X_test, y_test)}")
avrg = ['micro', 'macro', 'weighted', None]
for a in avrg:
    print(f"{a}: {f1_score(y_test, x_pred, average=a)}")

score: 0.9919903279431767
micro: 0.9919903279431767
macro: 0.9919899064429678
weighted: 0.9919905123495181
None: [0.9919318  0.99204801]


In [38]:
print(people.images.shape[1:3])
joblib.dump(cls, 'cls.pkl')

(62, 47)


['cls.pkl']